

## echo.cwlの生成

In [79]:
%%writefile cwls/echo.cwl

cwlVersion: v1.2
class: CommandLineTool
baseCommand: "echo"
hints:
    DockerRequirement:
        dockerPull: biocontainers/htseq:v0.11.2-1-deb-py3_cv1
stdout: hello.txt
inputs:
    inp1:
        type: string
        default: "Hello CWL"
        inputBinding:
            position: 1
    inp2:
        type: string
        default: "Hello jupyter"
        inputBinding:
            position: 2
outputs:
    output:
        type: stdout

Writing cwls/echo.cwl


In [83]:
!cwltool --validate cwls/echo.cwl

INFO /usr/local/bin/cwltool 3.1.20220907141119
INFO Resolved 'cwls/echo.cwl' to 'file:///workspace/cwls/echo.cwl'
cwls/echo.cwl is valid CWL.


## cwltool.factoryを用いたecho.cwlの実行

In [81]:
import cwltool
import cwltool.factory

fac = cwltool.factory.Factory()
print( fac.runtime_context.rm_tmpdir )
print( fac.runtime_context.outdir )

fac.runtime_context.rm_tmpdir = False
fac.runtime_context.outdir = "./test/"

echo = fac.make("./cwls/echo.cwl")
result = echo(inp1="hello", inp2="test")

Resolved './cwls/echo.cwl' to 'file:///workspace/cwls/echo.cwl'
[job echo.cwl_5] /tmp/g3lpiik8$ docker \
    run \
    -i \
    --mount=type=bind,source=/tmp/g3lpiik8,target=/BPNEWS \
    --mount=type=bind,source=/tmp/fbmdvm0p,target=/tmp \
    --workdir=/BPNEWS \
    --read-only=true \
    --net=none \
    --log-driver=none \
    --user=0:0 \
    --rm \
    --env=TMPDIR=/tmp \
    --env=HOME=/BPNEWS \
    biocontainers/htseq:v0.11.2-1-deb-py3_cv1 \
    echo \
    hello \
    test > /tmp/g3lpiik8/hello.txt


True
None


[job echo.cwl_5] completed success


In [58]:
!cat ./test/hello.txt

hello test


## seqkit_fx2tab.cwlの実行

In [16]:
!cwltool --validate ./cwls/seqkit_fx2tab.cwl

INFO /usr/local/bin/cwltool 3.1.20220907141119
INFO Resolved './cwls/seqkit_fx2tab.cwl' to 'file:///workspace/cwls/seqkit_fx2tab.cwl'
./cwls/seqkit_fx2tab.cwl is valid CWL.


In [17]:
## 失敗例：fastxに渡すパラメータが文字列のため、エラーを吐く
import cwltool
import cwltool.factory

fac = cwltool.factory.Factory()
print( fac.runtime_context.rm_tmpdir )
print( fac.runtime_context.outdir )

fac.runtime_context.rm_tmpdir = False
fac.runtime_context.outdir = "./result_seqkit/"
seqkit_1 = fac.make("./cwls/seqkit_fx2tab.cwl")
result = seqkit_1( fastx="./data/il7.fa" )


Resolved './cwls/seqkit_fx2tab.cwl' to 'file:///workspace/cwls/seqkit_fx2tab.cwl'


True
None


WorkflowException: Invalid job input record:
the `fastx` field is not valid because
  is not a dict

In [25]:
## 成功例：fastxにFile objectを渡す.

import cwltool
import cwltool.factory

fac = cwltool.factory.Factory()
print( fac.runtime_context.rm_tmpdir )
print( fac.runtime_context.outdir )

fac.runtime_context.rm_tmpdir = False
fac.runtime_context.outdir = "./result_seqkit/"

seqkit_1 = fac.make("./cwls/seqkit_fx2tab.cwl")
# fastx_obj = cwltool.hogehoge
result = seqkit_1(outputname="hoge.tsv", fastx= {"class": "File", "path":"./data/il7.fa","basename":"il7.fa", "contents":""} )



Resolved './cwls/seqkit_fx2tab.cwl' to 'file:///workspace/cwls/seqkit_fx2tab.cwl'
[job seqkit_fx2tab.cwl_12] /tmp/mh4pox1s$ docker \
    run \
    -i \
    --mount=type=bind,source=/tmp/mh4pox1s,target=/WeXvjW \
    --mount=type=bind,source=/tmp/jawsc2l4,target=/tmp \
    --mount=type=bind,source=/tmp/vvp4wivi/il7.fa,target=/var/lib/cwl/stg791eb8af-eb50-46b5-85a4-dc9d13213752/il7.fa,readonly \
    --workdir=/WeXvjW \
    --read-only=true \
    --log-driver=none \
    --user=0:0 \
    --rm \
    --env=TMPDIR=/tmp \
    --env=HOME=/WeXvjW \
    pegi3s/seqkit:2.3.0 \
    fx2tab \
    /var/lib/cwl/stg791eb8af-eb50-46b5-85a4-dc9d13213752/il7.fa > /tmp/mh4pox1s/hoge.tsv


True
None


[job seqkit_fx2tab.cwl_12] completed success


In [28]:
!cat ./result_seqkit/hoge.tsv

- 何故かカラ
- 良くわからないので、ここで諦める

- cwlに置いて、ファイルオボジェクトがどう表現されているか、cwltool の中にobjectがあるはず。